In [25]:
#importing library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import r2_score

In [26]:
#loading the dataset
df=pd.read_csv('../input/sales-forecasting/train.csv')
#loading the dataset
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,9796,CA-2017-125920,21/05/2017,28/05/2017,Standard Class,SH-19975,Sally Hughsby,Corporate,United States,Chicago,Illinois,60610.0,Central,OFF-BI-10003429,Office Supplies,Binders,"Cardinal HOLDit! Binder Insert Strips,Extra St...",3.7980
9796,9797,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,OFF-AR-10001374,Office Supplies,Art,"BIC Brite Liner Highlighters, Chisel Tip",10.3680
9797,9798,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10004977,Technology,Phones,GE 30524EE4,235.1880
9798,9799,CA-2016-128608,12/01/2016,17/01/2016,Standard Class,CS-12490,Cindy Schnelling,Corporate,United States,Toledo,Ohio,43615.0,East,TEC-PH-10000912,Technology,Phones,Anker 24W Portable Micro USB Car Charger,26.3760


In [27]:
#getting information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

In [36]:
#Encoding dates
def encode_dates(df,column):
    #copying the dataframe
    df=df.copy()
    #converting the column to date time
    df[column]=pd.to_datetime(df[column])
    #extracting year
    df[column+'_year']=df[column].apply(lambda x:x.year)
    #extracting month
    df[column+'_month']=df[column].apply(lambda x:x.month)
    #extracting day
    df[column+'_day']=df[column].apply(lambda x:x.day)
    #droping the original column
    df=df.drop(column,axis=1)
    #returning the dataframe
    return df

In [54]:
def onehot_encode(df,column):
    #copying the data
    df=df.copy()
    #getting dummies column and storing it in a variable
    dummies=pd.get_dummies(df[column],prefix=column)
    #concating the dummies columns with original columns
    df=pd.concat([df,dummies],axis=1)
    #droppping the original columnn
    df=df.drop(column,axis=1)
    #returning the dataframe
    return df

In [69]:
#Preprocessing 
def preprocess_inputs(df):
    #copying the dataset
    df=df.copy()
    #Dropping the unnecessary columns
    #dropping the customer id
    df=df.drop(['Row ID','Customer Name','Country','Product Name'],axis=1)
    #returning the dataset
    #Drop customer specific features columns
    df=df.drop(['Order ID','Customer ID'],axis=1)
    #Extracting date features
    df=encode_dates(df,column='Order Date')
    df=encode_dates(df,column='Ship Date')
    #one-hot encode 
    for column in ['Ship Mode','Segment','City','State','Postal Code','Region','Product ID','Category','Sub-Category']:
        df=onehot_encode(df,column=column)
    #Splitting df into x and y
    y=df['Sales']
    x=df.drop('Sales',axis=1)
    #train_test_split
    x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7,shuffle=True,random_state=1)
    #scaling the dataset
    scaler=StandardScaler()
    scaler.fit(x_train)
    x_train=pd.DataFrame(scaler.transform(x_train),columns=x_train.columns,index=x_train.index)
    x_test= pd.DataFrame(scaler.transform(x_test),columns=x_test.columns,index=x_test.index)   
    
    
    return x_train,x_test,y_train,y_test

In [70]:
x_train,x_test,y_train,y_test=preprocess_inputs(df)

In [71]:
x_train

,Order Date_year,Order Date_month,Order Date_day,Ship Date_year,Ship Date_month,Ship Date_day,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class,...,Sub-Category_Envelopes,Sub-Category_Fasteners,Sub-Category_Furnishings,Sub-Category_Labels,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables
8788,1.153020,-0.351009,-0.142358,1.138089,-0.387963,0.315968,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
847,-0.626244,0.817035,-0.389593,-0.638920,0.792139,0.076408,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
3397,1.153020,-1.811064,-1.131297,1.138089,0.202088,-1.121394,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
6244,-0.626244,1.109046,0.475728,-0.638920,1.087165,1.034650,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
8198,1.153020,-0.643020,-0.884062,1.138089,0.497114,-0.881834,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,1.153020,0.817035,0.846580,1.138089,0.792139,1.513771,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,-0.400194,3.178101,-0.30333,-0.13789,-0.179443
7813,0.263388,-1.519053,-0.265975,0.249584,-1.568065,-0.163153,2.326202,-0.249166,-0.484918,-1.212556,...,-0.162742,-0.145387,3.064953,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443
905,0.263388,0.233013,-1.502149,0.249584,1.382191,-1.480735,-0.429885,-0.249166,-0.484918,0.824704,...,-0.162742,-0.145387,-0.326269,-0.187924,-0.108625,2.498785,-0.314653,-0.30333,-0.13789,-0.179443
5192,-1.515875,1.109046,1.464667,-1.527425,-1.863091,-0.522493,-0.429885,-0.249166,2.062203,-1.212556,...,-0.162742,-0.145387,3.064953,-0.187924,-0.108625,-0.400194,-0.314653,-0.30333,-0.13789,-0.179443


In [73]:
x_train.shape[1]

3102

In [76]:
#Training the dataset
inputs=tf.keras.Input(shape=(x_train.shape[1],))
x=tf.keras.layers.Dense(128,activation='relu')(inputs)
x=tf.keras.layers.Dense(128,activation='relu')(x)
outputs=tf.keras.layers.Dense(1,activation='linear')(x)
model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 3102)]            0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               397184    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 413,825
Trainable params: 413,825
Non-trainable params: 0
_________________________________________________________________


In [78]:
model.compile(
optimizer='adam',
loss='mse')
history=model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau()
    ])


2022-05-24 14:34:54.607469: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
172/172 [==============================] - 1s 5ms/step - loss: 500766.5938 - val_loss: 197475.1562
Epoch 2/100
172/172 [==============================] - 1s 4ms/step - loss: 414607.5938 - val_loss: 156477.7812
Epoch 3/100
172/172 [==============================] - 1s 4ms/step - loss: 336682.2500 - val_loss: 161804.7500
Epoch 4/100
172/172 [==============================] - 1s 4ms/step - loss: 317572.8125 - val_loss: 487364.1562
Epoch 5/100
172/172 [==============================] - 1s 4ms/step - loss: 281858.4375 - val_loss: 872721.7500
Epoch 6/100
172/172 [==============================] - 1s 4ms/step - loss: 204749.4531 - val_loss: 2872936.0000
Epoch 7/100
172/172 [==============================] - 1s 4ms/step - loss: 157409.8906 - val_loss: 8336059.5000


In [80]:
test_loss=model.evaluate(x_test,y_test,verbose=0)

92/92 [==============================] - 0s 2ms/step - loss: 231599.3281


231599.328125

In [81]:
y_pred=np.squeeze(model.predict(x_test))

In [82]:
test_r2=r2_score(y_test,y_pred)

In [83]:
print('Test R^2 Score :{:.5f}'.format(test_r2))

Test R^2 Score :0.20021


In [66]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(6860, 3102)
(2940, 3102)
(6860,)
(2940,)


In [57]:
#checking the 
{column:len(x[column].unique()) for column in x.columns}

{'Sales': 5757,
 'Order Date_year': 4,
 'Order Date_month': 12,
 'Order Date_day': 31,
 'Ship Date_year': 5,
 'Ship Date_month': 12,
 'Ship Date_day': 31,
 'Ship Mode_First Class': 2,
 'Ship Mode_Same Day': 2,
 'Ship Mode_Second Class': 2,
 'Ship Mode_Standard Class': 2,
 'Segment_Consumer': 2,
 'Segment_Corporate': 2,
 'Segment_Home Office': 2,
 'City_Aberdeen': 2,
 'City_Abilene': 2,
 'City_Akron': 2,
 'City_Albuquerque': 2,
 'City_Alexandria': 2,
 'City_Allen': 2,
 'City_Allentown': 2,
 'City_Altoona': 2,
 'City_Amarillo': 2,
 'City_Anaheim': 2,
 'City_Andover': 2,
 'City_Ann Arbor': 2,
 'City_Antioch': 2,
 'City_Apopka': 2,
 'City_Apple Valley': 2,
 'City_Appleton': 2,
 'City_Arlington': 2,
 'City_Arlington Heights': 2,
 'City_Arvada': 2,
 'City_Asheville': 2,
 'City_Athens': 2,
 'City_Atlanta': 2,
 'City_Atlantic City': 2,
 'City_Auburn': 2,
 'City_Aurora': 2,
 'City_Austin': 2,
 'City_Avondale': 2,
 'City_Bakersfield': 2,
 'City_Baltimore': 2,
 'City_Bangor': 2,
 'City_Bartlett':

In [58]:
x

,Sales,Order Date_year,Order Date_month,Order Date_day,Ship Date_year,Ship Date_month,Ship Date_day,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,...,Sub-Category_Envelopes,Sub-Category_Fasteners,Sub-Category_Furnishings,Sub-Category_Labels,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables
0,261.9600,2017,8,11,2017,11,11,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,731.9400,2017,8,11,2017,11,11,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,14.6200,2017,12,6,2017,6,16,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,957.5775,2016,11,10,2016,10,18,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,22.3680,2016,11,10,2016,10,18,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9795,3.7980,2017,5,21,2017,5,28,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9796,10.3680,2016,12,1,2016,1,17,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9797,235.1880,2016,12,1,2016,1,17,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9798,26.3760,2016,12,1,2016,1,17,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [45]:
x['Ship Mode'].unique()

array(['Second Class', 'Standard Class', 'First Class', 'Same Day'],
      dtype=object)

In [47]:
pd.get_dummies(x['Ship Mode'],prefix='mode')

,mode_First Class,mode_Same Day,mode_Second Class,mode_Standard Class
0,0,0,1,0
1,0,0,1,0
2,0,0,1,0
3,0,0,0,1
4,0,0,0,1
...,...,...,...,...
9795,0,0,0,1
9796,0,0,0,1
9797,0,0,0,1
9798,0,0,0,1
